In [1]:
from pprint import pprint
import pandas as pd
import time
import IPython
pd.options.mode.chained_assignment = None  # default='warn'

# Opportunities

In [2]:
df = pd.DataFrame(pd.read_csv("../logs/arbbot.csv", error_bad_lines=False))
df["humanTimestamp"] = pd.to_datetime(df['blockTimestamp'], unit='s')
df["runTime"] = df["endTimestamp"] - df["startTimestamp"]

In [9]:
# Limit to hour span
hour_limit = 48
sec_limit = hour_limit * 3600
df = df[df.blockTimestamp > time.time()-sec_limit]

In [10]:
# df[df["netProfit"]>0].sort_values(by="netProfit", ascending=0)
under10 = df[(df.netProfit>0)&(df.optimalAmount<10)].sort_values(by="netProfit", ascending=0)
print(f"Amount of opp with <10eth input (24h): {len(under10)}")
print(f"Average input amount (24h): {under10.optimalAmount.mean()} eth")
print(f"Average net profit (24h): {under10.netProfit.mean()} eth")

Amount of opp with <10eth input (24h): 154
Average input amount (24h): 2.989592207792208 eth
Average net profit (24h): 0.013639262792207793 eth


In [11]:
# df[df["netProfit"]>0].sort_values(by="netProfit", ascending=0)
above10 = df[(df.netProfit>0)&(df.optimalAmount>10)].sort_values(by="netProfit", ascending=0)
print(f"Amount of opp ({hour_limit}h): {len(above10)}")
print(f"Average input amount ({hour_limit}h): {above10.optimalAmount.mean()} eth")
print(f"Average net profit ({hour_limit}h): {above10.netProfit.mean()} eth")

Amount of opp (48h): 17
Average input amount (48h): 28.321764705882355 eth
Average net profit (48h): 0.05557961764705882 eth


## Best opportunities so far

In [8]:
df.sort_values(by="netProfit", ascending=0).head(5)

,blockNumber,blockTimestamp,instrName,instrId,optimalAmount,grossProfit,netProfit,gasCost,gasAmount,gasAmountArch,startTimestamp,endTimestamp,humanTimestamp,runTime
102971,11606233,1610004949,weth2yfi2weth_sushiswap2uniswap,I0012,37.520,0.15590,0.13610,0.01979,179907,264907.0,1.610005e+09,1.610005e+09,2021-01-07 07:35:49,2.58564
94859,11604505,1609982092,weth2band2weth_sushiswap2uniswap,I0018,7.409,0.14540,0.11010,0.03533,247092,384078.0,1.609982e+09,1.609982e+09,2021-01-07 01:14:52,1.70676
89208,11602591,1609956439,weth2sushi2weth_sushiswap2Uniswap,I0008,7.722,0.12270,0.09345,0.02926,240000,NaN,1.609956e+09,1.609956e+09,2021-01-06 18:07:19,3.09957
89212,11602592,1609956447,weth2sushi2weth_sushiswap2Uniswap,I0008,7.722,0.12270,0.09345,0.02926,240000,NaN,1.609956e+09,1.609956e+09,2021-01-06 18:07:27,2.95974
103335,11606368,1610006765,weth2yfi2weth_sushiswap2uniswap,I0012,25.320,0.07105,0.05467,0.01637,179907,264907.0,1.610007e+09,1.610007e+09,2021-01-07 08:06:05,1.22910


## Statistics

In [9]:
# Limit opportuninities to the ones beyond the threshold to account for gas costs
net_profitable_opps = df[df.netProfit > 0]
opp_count = net_profitable_opps.blockNumber.count()
runtime_mean = net_profitable_opps.runTime.mean()
max_profit = net_profitable_opps.netProfit.max()
mean_profit = net_profitable_opps.netProfit.mean()
total_profit = net_profitable_opps.netProfit.sum()

In [10]:
print(f"Opportunitys count: {opp_count}")
print(f"Latency average: {runtime_mean:.2f} sec")
print(f"Max profit: {max_profit:.2f} ETH")
print(f"Average profit: {mean_profit:.2f} ETH")

Opportunitys count: 97
Latency average: 1.87 sec
Max profit: 0.14 ETH
Average profit: 0.01 ETH


## Instruction analysis

In [14]:
opp_comp_df = net_profitable_opps.groupby("instrName").agg({"blockNumber": "count", "netProfit": ["max", "sum", "mean"]})
opp_comp_df.columns = ["count", "max_profit", "summed_profit", "mean_profit"]
opp_comp_df.index.name = None
opp_comp_df.sort_values(by="mean_profit", ascending=0)

,count,max_profit,summed_profit,mean_profit
weth2sushi2weth_sushiswap2Uniswap,2,0.093450,0.186900,0.093450
weth2yfi2weth_sushiswap2uniswap,6,0.136100,0.307614,0.051269
weth2band2weth_uniswap2sushiswap,1,0.042390,0.042390,0.042390
weth2dai2weth_sushiswap2Uniswap,1,0.024150,0.024150,0.024150
weth2band2weth_sushiswap2uniswap,7,0.110100,0.139049,0.019864
weth2inj2weth_uniswap2sushiswap,2,0.014810,0.029620,0.014810
weth2wbtc2weth_uniswap2Sushiswap,3,0.019410,0.042870,0.014290
weth2kp3r2weth_sushiswap2uniswap,4,0.019030,0.056996,0.014249
weth2kp3rweth_sushiswap2Uniswap,2,0.015000,0.026880,0.013440
weth2usdc2weth_uniswap2sushiswap,1,0.012640,0.012640,0.012640


In [19]:
gross_profit_instr = df.groupby("instrName").agg({"blockNumber": "count", "grossProfit": ["max", "sum", "mean"]})
gross_profit_instr.columns = ["count", "max_profit", "summed_profit", "mean_profit"]
gross_profit_instr.index.name = None
gross_profit_instr.sort_values(by="mean_profit", ascending=0)

,count,max_profit,summed_profit,mean_profit
weth2link2weth_sushiswap2Uniswap,3,0.024530,0.068140,0.022713
weth2link2weth_uniswap2Sushiswap,8,0.021660,0.114330,0.014291
weth2usdc2weth_sushiswap2uniswap,1,0.014250,0.014250,0.014250
weth2yfi2weth_sushiswap2uniswap,47,0.155900,0.583542,0.012416
weth2dai2weth_sushiswap2Uniswap,8,0.053430,0.097995,0.012249
weth2wbtc2weth_uniswap2sushiswap,5,0.017680,0.058074,0.011615
weth2yfi2weth_sushiswap2Uniswap,13,0.015500,0.128356,0.009874
weth2link2weth_uniswap2sushiswap,25,0.014030,0.201716,0.008069
weth2wnxm2weth_uniswap2sushiswap,1532,0.023620,10.894798,0.007111
weth2usdc2weth_uniswap2sushiswap,10,0.029030,0.064250,0.006425


# Archer submissions

In [12]:
df_arch = pd.DataFrame(pd.read_csv("../logs/archer_api_requests.csv"))
df_arch.columns = [
                "bot_id", 
                "target_block", 
                "trade", 
                "estimated_profit_before_gas", 
                "gas_estimate", 
                "query", 
                "query_breakeven",
                "input_amount",
                "input_asset",
                "query_insert_locations", 
                "trade_insert_locations", 
                "response"
             ]

In [13]:
pd.set_option('display.max_colwidth', -1)
cut = df_arch[df_arch.target_block==11602794]
if len(cut):
    cut = cut.iloc[0]
print(cut)

bot_id                         2                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

/home/mike/Desktop/mosquito/env/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


"0x6880608f00000000000000000000000000000000000000000000000000000000000000c0000000000000000000000000000000000000000000000000000000000000026000000000000000000000000000000000000000000000000000000000000002a0000000000000000000000000000000000000000000000000000000000000052000000000000000000000000000000000000000000000000029c9388c0ab7298000000000000000000000000000000000000000000000000029c9388c0ab380000000000000000000000000000000000000000000000000000000000000000170121835e15703a1a7bab32626d0927d60f90a81d700000000000000000000000000000000000000000000000000000000000000847f2b0c090000000000000000000000007a250d5630b4cf539739df2c5dacb4c659f2488d00000000000000000000000000000000000000000000000029c9388c0ab38000000000000000000000000000c02aaa39b223fe8d0a0e5c4f27ead9083c756cc20000000000000000000000006b3595068778dd592e39a122f4f5a5cf09c90fe2121835e15703a1a7bab32626d0927d60f90a81d700000000000000000000000000000000000000000000000000000000000000847f2b0c09000000000000000000000000d9e1ce17f2641f24ae83637ab66a2cca9c378b9f00000000000000000000000000000000000000000000000fee1dead0fbadf00d0000000000000000000000006b3595068778dd592e39a122f4f5a5cf09c90fe2000000000000000000000000c02aaa39b223fe8d0a0e5c4f27ead9083c756cc2000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000011000000000000000000000000000000000000000000000000000000000000002507a250d5630b4cf539739df2c5dacb4c659f2488d00000000000000000000000000000000000000000000000000000000000000e47ff36ab500000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000080000000000000000000000000d30ce37a6f2424593dabe9b712d235781815445d000000000000000000000000000000000000000000000000000000005ff4ff120000000000000000000000000000000000000000000000000000000000000002000000000000000000000000c02aaa39b223fe8d0a0e5c4f27ead9083c756cc20000000000000000000000006b3595068778dd592e39a122f4f5a5cf09c90fe2d9e1ce17f2641f24ae83637ab66a2cca9c378b9f000000000000000000000000000000000000000000000000000000000000010418cbafe500000000000000000000000000000000000000000000000fee1dead0fbadf00d000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000a0000000000000000000000000d30ce37a6f2424593dabe9b712d235781815445d000000000000000000000000000000000000000000000000000000005ff4ff1200000000000000000000000000000000000000000000000000000000000000020000000000000000000000006b3595068778dd592e39a122f4f5a5cf09c90fe2000000000000000000000000c02aaa39b223fe8d0a0e5c4f27ead9083c756cc20000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000150"